In [2]:
# !pip install pandas nltk numpy networkx bert-extractive-summarizer

In [8]:
import os
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk import tokenize
import numpy as np
import networkx as nx

from timeit import default_timer as timer
from datetime import timedelta

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/intern/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
data = pd.read_csv("consolidated_report_threatdetails.csv")

In [6]:
data

,filename,threatdetails,report_type
0,19-00002004.txt,Most Active Threats Threat Behavior Cyber Espi...,Country
1,19-00010134.txt,Threat Category Groups and TTPs Cyber Espionag...,Country
2,19-00016187.txt,Threat Category Groups and TTPs Cyber Espionag...,Country
3,19-00013939.txt,Threat Category Groups and TTPs Cyber Espionag...,Country
4,19-00012510.txt,Threat Category Groups and TTPs Cyber Crime Br...,Country
...,...,...,...
160,19-00012202.txt,"Campaign Details On July 12, 2019, FireEye Thr...",Campaign
161,15-00004940.txt,"New Version Details Version 2, July 16, 2019: ...",Campaign
162,19-00012194.txt,NOTE: This was published on the FireEye blog o...,Campaign
163,19-00012453.txt,Background Chanitor (aka Hancitor) is a downlo...,Campaign


# Textrank Extractive Summary

In [6]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1] #case insensitivity
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [7]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

In [8]:
def generate_textrank_summary(text, top_n=5):
    start = timer()
    
    stop_words = stopwords.words('english')
    
    # Step 1 - Sentence tokenization of 1 report text
    sentences =  tokenize.sent_tokenize(text)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    top_n_sentences = sorted([[scores[i], s] for i, s in enumerate(sentences)], reverse=True)[:top_n]    

    # Step 5 - Join the list of sentences up
    summary = " ".join(list(map(lambda x: x[1], top_n_sentences)))
    
    end = timer()
    
    return summary, str(timedelta(seconds=end-start))

In [9]:
data["textrank_extractive_summary"] = data["threatdetails"].apply(generate_textrank_summary)

In [10]:
temp = pd.DataFrame(data["textrank_extractive_summary"].values.tolist(), columns=["textrank_extractive_summary", "textrank_extractive_summary_time_taken"])

In [11]:
data["textrank_extractive_summary"] = temp["textrank_extractive_summary"]
data["textrank_extractive_summary_time_taken"] = temp["textrank_extractive_summary_time_taken"]

In [12]:
del temp

In [13]:
data.head()

,filename,threatdetails,report_type,textrank_extractive_summary,textrank_extractive_summary_time_taken
0,19-00002004.txt,Most Active Threats Threat Behavior Cyber Espi...,Country,"Domestically, a couple operating a seditious w...",0:00:03.787680
1,19-00010134.txt,Threat Category Groups and TTPs Cyber Espionag...,Country,We surmise that decreased Russian information ...,0:00:10.079509
2,19-00016187.txt,Threat Category Groups and TTPs Cyber Espionag...,Country,"Close engagements with the U.S., South Korea's...",0:00:05.695461
3,19-00013939.txt,Threat Category Groups and TTPs Cyber Espionag...,Country,APT41 financially motivated operations primari...,0:00:14.230108
4,19-00012510.txt,Threat Category Groups and TTPs Cyber Crime Br...,Country,Russia Although FireEye Threat Intelligence ha...,0:00:05.538054


# BERT Extractive Summary

https://github.com/dmmiller612/bert-extractive-summarizer

In [2]:
from summarizer import Summarizer
from time import sleep

2022-07-21 11:28:30.082744: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 11:28:30.088458: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-07-21 11:28:30.088693: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
body = '''
The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price.
The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal.
Mubadala, an Abu Dhabi investment fund, purchased 90% of the building for $800 million in 2008.
Real estate firm Tishman Speyer had owned the other 10%.
The buyer is RFR Holding, a New York real estate company.
Officials with Tishman and RFR did not immediately respond to a request for comments.
It's unclear when the deal will close.
The building sold fairly quickly after being publicly placed on the market only two months ago.
The sale was handled by CBRE Group.
The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.
The rent is rising from $7.75 million last year to $32.5 million this year to $41 million in 2028.
Meantime, rents in the building itself are not rising nearly that fast.
While the building is an iconic landmark in the New York skyline, it is competing against newer office towers with large floor-to-ceiling windows and all the modern amenities.
Still the building is among the best known in the city, even to people who have never been to New York.
It is famous for its triangle-shaped, vaulted windows worked into the stylized crown, along with its distinctive eagle gargoyles near the top.
It has been featured prominently in many films, including Men in Black 3, Spider-Man, Armageddon, Two Weeks Notice and Independence Day.
The previous sale took place just before the 2008 financial meltdown led to a plunge in real estate prices.
Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest.
Blackstone Group (BX) bought it for $1.3 billion 2015.
The Chrysler Building was the headquarters of the American automaker until 1953, but it was named for and owned by Chrysler chief Walter Chrysler, not the company itself.
Walter Chrysler had set out to build the tallest building in the world, a competition at that time with another Manhattan skyscraper under construction at 40 Wall Street at the south end of Manhattan. He kept secret the plans for the spire that would grace the top of the building, building it inside the structure and out of view of the public until 40 Wall Street was complete.
Once the competitor could rise no higher, the spire of the Chrysler building was raised into view, giving it the title.
'''

model = Summarizer()
result = model(body, num_sentences=6)
full = ''.join(result)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [4]:
full

'The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price. The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal. Real estate firm Tishman Speyer had owned the other 10%. The buyer is RFR Holding, a New York real estate company. The building sold fairly quickly after being publicly placed on the market only two months ago. He kept secret the plans for the spire that would grace the top of the building, building it inside the structure and out of view of the public until 40 Wall Street was complete.'

In [12]:
def generate_bert_summary(text, num_sentences=6): 
    #63 is chosen because "Please rate this product by taking a short four question survey" is 63 characters.
    #removes that line from the output
    #can probably tune it without using such hacky method
    start = timer()
    
    model = Summarizer()
    sleep(100)
    result = model(text, num_sentences=num_sentences)
    summary = ''.join(result)
    
    end = timer()

    return summary, str(timedelta(seconds=end-start))

In [13]:
data = pd.read_csv(os.path.join("assets","ruscraper_results.csv"))

In [14]:
data

,Reports,TTP,Actor
0,Mandiant Threat Intelligence assesses with mo...,"['Malware Propagation and Deployment', 'Doxing...",TEMP.Isotope
1,The cyber espionage threat to the manufacturi...,['No ttp provided.'],TEMP.Isotope
2,"As of December 2017, FireEye iSIGHT Intellige...","['Malware Propagation and Deployment', 'Networ...",TEMP.Isotope
3,Possible Russia-nexus Sandworm Team targeting...,"['Malware Propagation and Deployment', 'Networ...",TEMP.Isotope
4,Given the continued escalations in geopolitic...,"['Malware Propagation and Deployment', 'Networ...",TEMP.Isotope
...,...,...,...
1149,Mandiant Threat Intelligence identified a new...,"['Social Engineering', 'Malware Propagation an...",UNC1151
1150,"In mid-2021, Mandiant Threat Intelligence unc...",['No ttp provided.'],UNC1151
1151,"In late May 2021, Mandiant Threat Intelligenc...",['Domain Registration_DNS Abuse and Manipulati...,UNC1151
1152,"On March 19, 2022, spear-phishing emails from...",['No ttp provided.'],UNC1151


In [17]:
data["Summary"] = data["Reports"].apply(generate_bert_summary)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
temp = pd.DataFrame(data["Summary"].values.tolist(), columns=["Summary", "Summary_time_taken"])

In [ ]:
data["Summary"] = temp["Summary"]
data["Summary_time_taken"] = temp["Summary_time_taken"]

In [ ]:
del temp

In [ ]:
data.to_csv("summary_of_reports_cpuonly.csv", index=False)

In [ ]:
import random
random.randint(1, len(data))

In [ ]:
data.iloc[71]["Summary_time_taken"]

In [ ]:
data["Summary_time_taken"]

In [ ]:
data_gpu = pd.read_csv("summary_of_reports.csv")

In [ ]:
data_gpu.iloc[71]["Summary_time_taken"]

In [ ]:
import datetime
import pytimeparse
import unittest

def reconstruct_timedelta(td_string):
    seconds = pytimeparse.parse(td_string)
    return datetime.timedelta(seconds=seconds).total_seconds()

In [ ]:
np.mean(list(map(lambda x: reconstruct_timedelta(x), data["Summary_time_taken"])))

In [ ]:
np.mean(list(map(lambda x: reconstruct_timedelta(x), data_gpu["Summary_time_taken"])))